Part 1

In [1]:
class Gate():
    def __init__(self, x, gate_type, y, z):
        assert gate_type in {'AND', 'OR', 'XOR'}

        self.executed = False
        self.x = x
        self.type = gate_type
        self.y = y
        self.z = z
    
    def __str__(self):
        return f'{self.x} {self.type} {self.y} -> {self.z}'
    
    def execute(self, registers):
        match self.type:
            case 'AND':
                registers[self.z] = registers[self.x] & registers[self.y]
            case 'OR':
                registers[self.z] = registers[self.x] | registers[self.y]
            case 'XOR':
                registers[self.z] = registers[self.x] ^ registers[self.y]

        self.executed = True
        return registers

def pt1(filename):
    with open(filename, 'r') as f:
        reg_strs, gate_strs = (x.splitlines() for x in f.read().strip().split('\n\n'))

    registers = dict()
    for rs in reg_strs:
        reg, val = rs.split(': ')
        registers[reg] = int(val)

    gates = []
    for gs in gate_strs:
        x, gate_type, y, _, z = gs.split()
        gates.append(Gate(x, gate_type, y, z))

    while not all(g.executed for g in gates):
        for g in gates:
            if g.executed:
                continue
            if g.x in registers and g.y in registers:
                registers = g.execute(registers)

    bstr = ''.join(str(registers[k]) for k in sorted(registers, reverse=True) if 'z' in k)

    return int(bstr, 2)

pt1('test.txt'), pt1('test2.txt'), pt1('input.txt')

(4, 2024, 65635066541798)

Part 2

In [2]:
def pt2(filename):
    with open(filename, 'r') as f:
        reg_strs, gate_strs = (x.splitlines() for x in f.read().strip().split('\n\n'))

    registers = dict()
    for rs in reg_strs:
        reg, val = rs.split(': ')
        registers[reg] = int(val)

    gates = []
    for gs in gate_strs:
        x, gate_type, y, _, z = gs.split()
        x, y = sorted((x, y))
        gates.append(Gate(x, gate_type, y, z))

    max_z = max(g.z for g in gates)

    swapped_wires = set()
    for g in gates:
        wrong_output = all(['z' in g.z, g.type != 'XOR', g.z != max_z])
        wrong_output |= all(s[0] not in 'xyz' for s in [g.x, g.y, g.z]) and g.type == 'XOR'
        for subg in gates:
            if g.z in (subg.x, subg.y):
                wrong_output |= g.type == 'AND' and subg.type != 'OR' and 'x00' not in [g.x, g.y]
                wrong_output |= g.type == 'XOR' and subg.type == 'OR'

        if wrong_output:
            swapped_wires.add(g.z)

    return ','.join(sorted(swapped_wires))

pt2('testpt2.txt'), pt2('input.txt')

('z00,z01,z02,z03,z04', 'dgr,dtv,fgc,mtj,vvm,z12,z29,z37')